# Как научить компьютер читать?



## 1. Подготовка и обучение

Будем обучать w2v модель на википедии. К счастью, в её случае для всех языков предусмотрена система дампов. С [удобной странчки](https://dumps.wikimedia.org) можно скачать текущую полную версию википедийного текста на любом языке. Например, [на русском.](https://dumps.wikimedia.org/ruwiki/).

Для обучения модели будем использовать библиотеку `gensim`. В ней уже есть удобный модуль доя работы с википедийными дампами, а также готовая хорошая реализация w2v-сетки. 

In [1]:
path = "/Users/fulyankin/Yandex.Disk.localized/Научные проекты/w2v/ruwiki-20180320-pages-articles-multistream.xml.bz2"

In [2]:
from gensim.corpora.wikicorpus import WikiCorpus

# для доступа к текстам мы будем пользоваться генератором wiki.get_texts()
wiki = WikiCorpus(path, dictionary=False)

Корпус википедии, оказавшийся в наших руках уже прошёл очистку от мусора и был токенезирован. Посмотрим на первые $40$ слов самой первой её статьи. 

In [3]:
i = 0
for text in wiki.get_texts( ):
    i+=1
    if i == 2:
        break
    else:
        print(text[:40])

['литва', 'официальное', 'название', 'лито', 'вская', 'респу', 'блика', 'государство', 'расположенное', 'северной', 'европе', 'одна', 'из', 'стран', 'балтии', 'столица', 'страны', 'вильнюс', 'площадь', 'км²', 'протяжённость', 'севера', 'на', 'юг', 'км', 'запада', 'на', 'восток', 'км', 'население', 'составляет', 'человек', 'по', 'этим', 'показателям', 'является', 'крупнейшим', 'прибалтийским', 'государством', 'имеет']


Попробуем выделить в тексте основные биграммы. Будем рассматривать их в дальнейшем как цельные токены. Существует целый ряд алгоритмов, занимающихся этим. Обычно все они сводятся к поиску вероятностей совместного появления двух слов в тексте. 

Код ниже для большого корпуса текстов будет работать довольно долго, но зато на выходе мы получим словарь биграмм, который мы сможем впоследствии применять к потоку текстов командой `bigram_transformer`. 

In [4]:
%%time
from gensim.models.phrases import Phrases, Phraser

# хочется посмотреть на самые частые биграммы и использовать их при обучении как токены
bigram = Phrases(wiki.get_texts())
bigram_transformer = Phraser(bigram)

# генератор текстов с биграммами
def text_generator_bigram( ):
    for text in wiki.get_texts( ):
        yield bigram_transformer[[word for word in text]]

CPU times: user 34min 40s, sys: 1min 32s, total: 36min 13s
Wall time: 44min 6s


Посмотрим что нам будет выдавать такой генератор на примере первой статьи с википедии.

In [5]:
i = 0
for item in text_generator_bigram( ):
    i +=1 
    if i == 2:
        break
    else:
        print(item[:20])

['литва', 'официальное', 'название', 'лито_вская', 'респу_блика', 'государство', 'расположенное', 'северной', 'европе', 'одна', 'из', 'стран_балтии', 'столица', 'страны', 'вильнюс', 'площадь_км²', 'протяжённость', 'севера', 'на', 'юг']


По аналогии можно соорудить код для поиска самых частых триграмм. Например, триграммой будет словосочетание "по моему мнению" или "вторая мировая война".

In [7]:
%%time
trigram = Phrases(text_generator_bigram())
trigram_transformer = Phraser(trigram)

def text_generator_trigram():
    for text in wiki.get_texts():
        yield trigram_transformer[bigram_transformer[[word for word in text]]]

CPU times: user 1h 7min 39s, sys: 2min 25s, total: 1h 10min 4s
Wall time: 2h 10min 8s


In [8]:
i = 0
for item in text_generator_trigram( ):
    i +=1 
    if i == 2:
        break
    else:
        print(item[:20])

['литва', 'официальное_название', 'лито_вская', 'респу_блика_государство', 'расположенное', 'северной', 'европе', 'одна', 'из', 'стран_балтии', 'столица', 'страны', 'вильнюс', 'площадь_км²', 'протяжённость_севера', 'на', 'юг', 'км', 'запада', 'на']


На этом давайте закончим нашу подготовку и попробуем собрать и обучить модель. Конечно же учиться она будет довольно долго. Не факт, что на слабом компьютере она вообще выучится. 

In [9]:
%%time 
from gensim.models.word2vec import Word2Vec

# теперь сама модель
# size - размерность векторов 
# window - ширина окна контеста
# min_count - если слово встречается реже, для него не учим модель
model = Word2Vec(size=300, window=7, min_count=10, workers=4)

# строительство словаря, чтобы обучение шло быстрее
model.build_vocab(text_generator_trigram())

CPU times: user 1h 7min 3s, sys: 1min 57s, total: 1h 9min
Wall time: 1h 14min 53s


In [10]:
%%time
# обучение модели 
model.train(text_generator_trigram(), total_examples=model.corpus_count, epochs=model.iter)

CPU times: user 1h 43min 22s, sys: 5min 24s, total: 1h 48min 47s
Wall time: 1h 25min 34s


323437961

Обученную модель можно сохранить. Процесс её обучения довольно трудоёмок, не очень хочется его повторять по несколько раз.

In [11]:
# сохраним обученную модель
model.save('wiki_model')

По аналогии сохраним создатель триграмм. Он тоже учился довольно долго, а нам хотелось бы в дальнейшем его переиспользовать.

In [12]:
trigram_transformer.save('wiki_trigramm')

## 2. Игрушечки

Попробуем поисследовать линейные соотношения между векторами в получившемся семантическом пространстве. 

In [ ]:
import gensim

# подгрузим обученную модель, если вдруг мы сбросили скрипт
# model = gensim.models.Word2Vec.load('wiki_model')

Для начала посмотрим как выглядит слово в получившемся пространстве.

In [13]:
# вектор слова
model.wv['король'][:10]

array([ 0.4187494 ,  1.3197291 ,  0.5561069 , -2.674209  , -0.39070314,
       -1.7381759 ,  2.4423337 ,  1.1509248 ,  0.49805412,  1.0481958 ],
      dtype=float32)

Можно посмотреть насколько слова похожи между собой. Эта похожесть считается с помощью косинусной метрики.

In [22]:
# похожесть между словами
model.wv.similarity('мужчина', 'женщина')

0.5731954174129064

In [20]:
model.wv.similarity('чашка', 'стакан')

0.7277190300871142

In [25]:
model.wv.similarity('грядка', 'станок')

0.14659733109020415

Попробуем провернуть первое уравнение. 

$$ Король + Женшина - Мужчина = \quad ???$$


In [26]:
model.most_similar(positive=['женщина', 'король'], negative=['мужчина'])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('королева', 0.6225535869598389),
 ('империя', 0.5604485869407654),
 ('принцесса', 0.5506317615509033),
 ('императрица', 0.5310197472572327),
 ('король_ок_ок', 0.5229284763336182),
 ('правительница', 0.5220396518707275),
 ('королевская_семья', 0.5136938095092773),
 ('герцогиня', 0.5069049596786499),
 ('великий_король', 0.5020265579223633),
 ('инквизиция', 0.4958014190196991)]

$$ Москва + Франция - Россия = \quad ???$$

In [27]:
model.most_similar(positive=['москва', 'франция'], negative=['россия'])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('париж', 0.5632048845291138),
 ('жан', 0.5306471586227417),
 ('жак', 0.5089118480682373),
 ('пьер', 0.5062865018844604),
 ('французский', 0.5051255822181702),
 ('шарль', 0.503093957901001),
 ('марсель', 0.49368593096733093),
 ('париже', 0.4840250611305237),
 ('анри', 0.4825429916381836),
 ('франсуа', 0.47708338499069214)]

$$ Математик + Женшина - Мужчина = \quad ???$$

In [28]:
model.most_similar(positive=['математик', 'женщина'], negative=['мужчина'])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('филолог', 0.6791591048240662),
 ('получившая_степень_доктора', 0.665369987487793),
 ('лингвист', 0.6601260900497437),
 ('доктор_философии', 0.6564381122589111),
 ('доктор_филологических_наук', 0.6553531885147095),
 ('переводчица', 0.6419261693954468),
 ('поэтесса', 0.639553964138031),
 ('поэтесса_писательница', 0.6373691558837891),
 ('доктор_философских_наук', 0.6361896991729736),
 ('педагог_профессор', 0.6341406106948853)]

$$ Ваше \mbox{ } уравнение $$

In [ ]:
model.most_similar(positive=[' ', ' '], negative=[' '])

In [ ]:
model.most_similar(positive=[' ', ' '], negative=[' '])

In [ ]:
model.most_similar(positive=[' ', ' '], negative=[' '])

Давайте попробуем просто посмотреть на самые близкие к каким-то выражениям слова.

In [29]:
model.most_similar('вторая_мировая_война')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('первая_мировая_война', 0.8385074138641357),
 ('корейская_война', 0.71861332654953),
 ('холодная_война', 0.6932598948478699),
 ('третий_рейх', 0.6707549095153809),
 ('люфтваффе', 0.6602753400802612),
 ('межвоенный_период', 0.6552859544754028),
 ('третьего_рейха', 0.6543014049530029),
 ('война_судного_дня', 0.6508054137229919),
 ('семилетняя_война', 0.6481649875640869),
 ('первой_мировой', 0.6462584733963013)]

In [30]:
model.most_similar('яндекс')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('mail_ru', 0.8583327531814575),
 ('яндекса', 0.8314518928527832),
 ('google', 0.8099757432937622),
 ('сети_интернет', 0.8059083223342896),
 ('веб', 0.7968020439147949),
 ('сервис', 0.7944517731666565),
 ('хостинг', 0.7870197296142578),
 ('yandex_ru', 0.7709559798240662),
 ('открытый_доступ', 0.7646337747573853),
 ('аккаунт', 0.7615125179290771)]

In [31]:
model.most_similar('ниу_вшэ')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('высшей_школы_экономики', 0.895513653755188),
 ('высшая_школа_экономики', 0.8587247729301453),
 ('российского_гуманитарного_университета', 0.8499684929847717),
 ('вшэ', 0.8479135036468506),
 ('ранхигс', 0.8308689594268799),
 ('мпгу', 0.830706000328064),
 ('гу_вшэ', 0.8254082798957825),
 ('мгимо', 0.8184138536453247),
 ('спбгу', 0.8165745139122009),
 ('мгпу', 0.8162704110145569)]

In [32]:
model.most_similar('аристотель')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('аристотеля', 0.8488722443580627),
 ('стоиков', 0.8021489381790161),
 ('гегель', 0.7997527122497559),
 ('ориген', 0.7984192371368408),
 ('своих_трудах', 0.7923400402069092),
 ('лейбниц', 0.789554238319397),
 ('плутарх', 0.7830368280410767),
 ('цицерон', 0.7817286252975464),
 ('маймонид', 0.7758834362030029),
 ('тацит', 0.7727030515670776)]

In [33]:
model.most_similar('машина_времени')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('король_шут', 0.8701559901237488),
 ('агата_кристи', 0.8659921288490295),
 ('сплин', 0.8545758724212646),
 ('весёлые_ребята', 0.8407142162322998),
 ('чайф', 0.8382944464683533),
 ('ногу_свело', 0.8371785879135132),
 ('ночные_снайперы', 0.8325160145759583),
 ('наив', 0.8310174942016602),
 ('валерий_леонтьев', 0.8306418657302856),
 ('андрей_макаревич', 0.8283897638320923)]

Давайте посмотрим на то, что интересует вас!

In [ ]:
model.most_similar(' ')

In [ ]:
model.most_similar(' ')

In [ ]:
model.most_similar(' ')

Нужно понимать, что в выборке, на основе которой вы обучали модель могут быть различные "артефакты". Например, если мы обучались на корпусе новостей, мы можем неожиданно обнаружить, что к Индонезии очень близко землятрясение. Почему? Да просто потому что в корпусе все статьи, связанные с Индонезией упоминали недавнее землятрясение. С такми артефактами приходится бороться и переодически модель приходится переобучать. В ситуации выше машина времени является подобным артефактом модели.

### Полезные ссылки 

* [Предобученная w2v для английского языка](https://code.google.com/archive/p/word2vec)
